# 文本向量处理

In [17]:
import numpy as np
import random 
with open("cnews/cnews.train.txt", 'r', encoding="utf-8") as file:
    train = file.readlines()
with open("cnews/cnews.test.txt", 'r', encoding="utf-8") as file:
    test = file.readlines()

In [18]:
def exactua(data):
    random.shuffle(data) #数据集打乱
    x_data = []
    y_data = []
    for line in data:
        x_data.append(line.replace('\n', '').split('\t')[1])
        y_data.append(line.replace('\n', '').split('\t')[0])
    return x_data, y_data

In [19]:
x_train_data, y_train_data = exactua(train)
x_test_data, y_test_data = exactua(test)

In [20]:
# 1000条 训练集
# 100条 测试
x_train_data, y_train_data = x_train_data[:1000], y_train_data[:10000]
x_test_data, y_test_data= x_test_data[:100], y_test_data[:100]

In [21]:
def stopwordslist():
    import jieba 
    stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

In [22]:
stopwords = stopwordslist()

In [23]:
def token(x_train_data, x_test_data):
    #去停用词并进行结巴分词
    import jieba
    trainlists = []
    for i in x_train_data:
        word_list = [word for word in jieba.cut(i) if word not in stopwords]
        trainlists.append(' '.join(word_list))
    testlists = []
    for i in x_test_data:
        word_list = [word for word in jieba.cut(i) if word not in stopwords]
        testlists.append(' '.join(word_list))
    return trainlists, testlists

In [24]:
x_train, x_test = token(x_train_data, x_test_data)
len(x_train)
len(x_test)

100

In [25]:
def data2vec(x_train, x_test):
    from sklearn.feature_extraction.text import CountVectorizer

    count_vec=CountVectorizer(stop_words=stopwords) #创建词袋数据结构
    X_count_train = count_vec.fit_transform(x_train)  #<class 'scipy.sparse.csr.csr_matrix'>
    #train和test转化为特征向量
    X_count_train= X_count_train.toarray()
    X_count_test = count_vec.transform(x_test).toarray()   
    return X_count_train, X_count_test, dict(count_vec.vocabulary_)

In [26]:
x_train, x_test, wordDic = data2vec(x_train, x_test)

In [27]:
x_train.shape

(1000, 40938)

In [28]:
print ('vocabulary:\n\n')
count = 0 
for key,value in wordDic.items():
    print(key,value)
    count += 1
    if count > 10:
        break

vocabulary:


研究 30194
女人 15700
眼泪 30010
降低 39040
男人 29226
性欲 19636
美人卷 32474
珠帘 28772
深坐 27134
蛾眉 34340
泪痕 26631


In [29]:
def y2label(y_train, y_test):
    dic = {}
    count = 0
    for i in set(y_train):
        dic[i] = count 
        count += 1
    y_train, y_test = [dic.get(i) for i in y_train_data], [dic.get(i) for i in y_test_data]
    return y_train, y_test, dic

In [30]:
y_train, y_test, dicLabel = y2label(y_train_data, y_test_data)

In [31]:
dicLabel

{'时尚': 0,
 '财经': 1,
 '教育': 2,
 '娱乐': 3,
 '房产': 4,
 '家居': 5,
 '时政': 6,
 '科技': 7,
 '游戏': 8,
 '体育': 9}

# 感知器网络

In [32]:
def loadData():
    import numpy as np
    import random 
    with open("cnews/cnews.train.txt", 'r', encoding="utf-8") as file:
        train = file.readlines()
    with open("cnews/cnews.test.txt", 'r', encoding="utf-8") as file:
        test = file.readlines()
    x_train_data, y_train_data = exactua(train)
    x_test_data, y_test_data = exactua(test)    
    # 1000条 训练集
    # 100条 测试
    x_train_data, y_train_data = x_train_data[:1000], y_train_data[:1000]
    x_test_data, y_test_data= x_test_data[:100], y_test_data[:100]
    stopwords = stopwordslist()
    x_train, x_test = token(x_train_data, x_test_data)
    x_train, x_test, wordDic = data2vec(x_train, x_test)
    y_train, y_test, dicLabel = y2label(y_train_data, y_test_data)
    return x_train, y_train, x_test, y_test

In [33]:
x_train, y_train, x_test, y_test = loadData()

In [34]:
x_train.shape

(1000, 40760)

In [35]:
y_train = np.array(y_train)
y_train = y_train[:,np.newaxis]
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
a=enc.fit_transform(y_train)
y_train = a.toarray()


C:\software\anaconda\envs\py36tf14\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [36]:
y_train.shape

(10000, 10)

In [37]:
import numpy as np
import tensorflow as tf

In [50]:
X = tf.placeholder(tf.float32, [None, x_train.shape[1]], name='X_placeholder') 
Y = tf.placeholder(tf.int32, [None, 10], name='Y_placeholder')

In [51]:
# 网络参数
n_hidden_1 = 128 # 第1个隐层  ，节点个数为256
n_hidden_2 = 64 # 第2个隐层
n_input =x_train.shape[1]# 
n_classes = 10 # 

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W')
}

#偏置在隐层的时候和节点个数一致，在输出层和输出类别个数一致
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='bias')
}

In [52]:
#此函数是用来构建计算的神经网络，得出预测类别的得分
def multilayer_perceptron(x, weights, biases):
    # 第1个隐层，使用relu激活函数
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'], name='fc_1')
    layer_1 = tf.nn.relu(layer_1, name='relu_1')
    # 第2个隐层，使用relu激活函数
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'], name='fc_2')
    layer_2 = tf.nn.relu(layer_2, name='relu_2')
    # 输出层
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'], name='fc_3')
    return out_layer


In [53]:
pred = multilayer_perceptron(X, weights, biases)

In [54]:
learning_rate = 0.001
loss_all = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y, name='cross_entropy_loss')
loss = tf.reduce_mean(loss_all, name='avg_loss')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


In [55]:
init = tf.global_variables_initializer()

In [56]:
def shuffle_aligned_list(data):
    """Shuffle arrays in a list by shuffling each array identically."""
    num = data[0].shape[0]
    p = np.random.permutation(num)
    return [d[p] for d in data]

def batch_generator(data, batch_size, shuffle=True):
    """Generate batches of data.

    Given a list of array-like objects, generate batches of a given
    size by yielding a list of array-like objects corresponding to the
    same slice of each input.
    """
    if shuffle:
        data = shuffle_aligned_list(data)

    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size > len(data[0]):
            batch_count = 0

            if shuffle:
                data = shuffle_aligned_list(data)

        start = batch_count * batch_size
        end = start + batch_size
        batch_count += 1
        yield [d[start:end] for d in data]


In [57]:
batch_gen = batch_generator([x_train, y_train], 1, shuffle=True)

In [58]:
batch_x,batch_y = next(batch_gen)

In [59]:
batch_y.shape

(1, 10)

In [ ]:
y_test = np.array(y_test)
y_test = y_test[:,np.newaxis]
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
a=enc.fit_transform(y_test)
y_test = a.toarray()

In [68]:
#训练总轮数
training_epochs = 1
#一批数据大小
batch_size = 1
#信息展示的频度
display_step = 5

with tf.Session() as sess:
    sess.run(init)

    # 训练
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(x_train.shape[0]/batch_size)
        # 遍历所有的batches
        for i in range(total_batch):
            batch_x,batch_y = next(batch_gen)
            # 使用optimizer进行优化
            _, l = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            # 求平均的损失
            avg_loss += l / total_batch
        # 每一步都展示信息
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_loss))
    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # 计算准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: x_test, Y: y_test}))   #此处accuracy是Tensor，在运行计算时需要用到eval()
    # print("Accuracy:", sess.run(accuracy,feed_dict = {X: mnist.test.images, Y: mnist.test.labels}))    #也可以用sess.run()

Epoch: 0001 cost= 1199.979976016
Optimization Finished!
Accuracy: 0.04


内存限制，只做batch=1的测试